<a href="https://colab.research.google.com/github/0zz10/Operation-Mask-2020/blob/master/steps_instructions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Operation Mask 2020**: a customized end-to-end disease control system

# 1.Set Up

## 1.1 Project setup on GCP and AutoML API enable

After Setting up a Google Cloud Project, in this case **cvfinal34944** is my project name.
We initialize the an API Service Account and download a JSON Key for edge computing.

![alt text](https://drive.google.com/uc?id=1QEtTvv7TXuNxxFgQIQyOKGhU9WFH3-uR)

## 1.2 Raspberry Pi Hardware Setup

![*Source:robot-advance.com*](https://www.robot-advance.com/userfiles/www.robot-advance.com/images/raspberry-4-modele-b-4go.jpg)

Raspberry Pi 4 Model B
*Source:robot-advance.com*
![Raspberry Pi GPIO Header](https://www.raspberrypi-spy.co.uk/wp-content/uploads/2012/06/Raspberry-Pi-GPIO-Header-with-Photo.png)

Raspberry Pi GPIO Header

## 1.3a) Remote Control of Raspberry Pi via SSH and Microsoft Remote Destop

I am using both MacOS terminal for RaspberryPi command input and Microsoft Remote for documents sharing and actual screen display.

in Raspberry terminal:

In [0]:
# First, the Raspberry Pi needs to be fully updated
sudo apt-get update
sudo apt-get dist-upgrade

ifconfig
# The IP address under Wlan0 is usded for Microsoft Remote on MacOS, in this case IP is 192.168.1.73

# reset passwd if neccessary
sudo passwd

in MacOS terminal:

In [0]:
ssh pi@192.168.1.73
# input password on pi

whoami
# return pi, show access bridged

# Get ready for VSN remote control acess
sudo apt install xrdp -y

using Microsoft Remote Destop:

## 1.3b) Remote Control of Raspberry Pi via VNC View

First, **Download** VNC Viewer in your desktop for remote control

After my TF card burnt out. I chose to make an alternative approach to remote control via VNC.

Follow the instruction below.

https://www.youtube.com/watch?v=kPeb5IvZW_k
https://magpi.raspberrypi.org/articles/vnc-raspberry-pi

in Raspberry Pi Terminal

In [0]:
# First, the Raspberry Pi needs to be fully updated
sudo apt-get update
sudo apt-get dist-upgrade
sudo apt-get install gimp

ifconfig
# The IP address under Wlan0 is usded for Microsoft Remote on MacOS, in this case IP is 192.168.1.73

# reset passwd if neccessary
sudo passwd

###1.3b) 2) for Raspberry elder edition
This part is  not neccessary for Raspberry Pi 4B or higher because pi has built-in vnc applicaitons installed


In [0]:
#Credits to http://4dc5.com/2012/06/12/setting-up-vnc-on-raspberry-pi-for-mac-access/ for developing a guide to base this tutorial on.

sudo apt-get install tightvncserver

vncserver :2020   #creat password

sudo apt-get install netatalk

ifconfig
# The IP address under inet is usded for Microsoft Remote on MacOS, in this case inet IP is 192.168.1.73

INow from the Mac terminal

In [0]:
#open Finder, and hit ⌘K. Enter vnc://192.168.1.73 (custimized by ifconfig-wlan0-inet).
sudo apt-get install avahi-daemon
sudo update-rc.d avahi-daemon defaults

sudo vim /etc/avahi/services/afpd.service

In [0]:
<?xml version="1.0" standalone='no'?><!--*-nxml-*-->
<!DOCTYPE service-group SYSTEM "avahi-service.dtd">
<service-group>
   <name replace-wildcards="yes">%h</name>
   <service>
      <type>_afpovertcp._tcp</type>
      <port>548</port>
   </service>
</service-group>

In [0]:
sudo /etc/init.d/avahi-daemon restart

#Will show the Pi in the sidebar on your Mac by default.
cd /etc/init.d
sudo vim tightvncserver

Add the following content and change 1440x900 to 1920x1080:

In [0]:
#Change the permissions of the file you just created
sudo chmod +x tightvncserver

#kill the server
sudo pkill Xtightvnc

sudo /etc/init.d/tightvncserver start
cd /etc/init.d
sudo update-rc.d tightvncserver defaults

In [0]:
sudo vim /etc/avahi/services/rfb.service

and add this content:

In [0]:
<?xml version="1.0" standalone='no'?>
<!DOCTYPE service-group SYSTEM "avahi-service.dtd">
<service-group>
  <name replace-wildcards="yes">%h</name>
  <service>
    <type>_rfb._tcp</type>
    <port>5901</port>
  </service>
</service-group>

In [0]:
#Finally Step
sudo /etc/init.d/avahi-daemon restart

## 1.4 Picamera Preparation

install Python Imaging Library

In [0]:
python3 -m pip install --user Pillow

install Raspberry Pi Camera V2

In [0]:
python3 -m pip install --user picamera

testing camera pi

In [0]:
raspistill -o image.png
raspistill -o image.jpg

# 2.Data Acquisition & Modelling

## 2.1 Trainning Data Prepartion

## 2. Mask:a customized GCP AutoML trained

![alt text](https://drive.google.com/uc?id=1EM3C_z1UyuoPjn1lhz33w_ivMSIwOl4a)

## 2.3 Configure an I2C-non contact temperature sensor MLX90614

I2C: a useful bus that allows data exchange between microcontrollers and peripherals with a minimum of wiring.


The I2C peripheral is not turned on by default. To enable it, do the following.

Raspberry Pi Configuration via Desktop GUI
You can use the Desktop GUI by heading to the Pi Start Menu > Preferences > Raspberry Pi Configuration

A window will pop up with different tabs to adjust settings. What we are interested is the Interfaces tab. **Click on the tab and select Enable for I2C**. At this point, you can enable additional interfaces depending on your project needs. Click on the OK button to same.

![alt text](https://cdn.sparkfun.com/r/600-600/assets/learn_tutorials/4/4/9/Raspberry-Pi-Configuration-Enable-Interfaces.png)

For hardware wirings setup, connect **SDA to Pin 3** on Raspberry Pi, **SCL to Pin 5**, **VCC to Pin 1** (3.3V) and **GND to any ground pin** (I used Pin 09) on Raspberry Pi.
![alt text](https://cdn.sparkfun.com/assets/learn_tutorials/4/4/9/pin-labels-on-pi-ver2.png)
![alt text](https://images-na.ssl-images-amazon.com/images/I/51Ect0McNXL._AC_SX522_.jpg)

In [0]:
#in your raspberry terminal or ssh terminal, issue the following command, 0x5a indicates the data input address, MLX90614
i2cdetect -y 1

![alt text](https://drive.google.com/uc?id=16bqDfH5U5CjhffL3CZhxhIfUfh1PX_s9)

## 2.3 Building a Serverless Data Pipeline: IoT to Analytics

build a weather data pipeline that starts with an Internet of Things (IoT) device, utilizes a message queue to receive and deliver data, leverages a serverless function to move the data to a data warehouse and then create a dashboard that displays the information. A Raspberry Pi with a contactless temperature sensor (MLX90614) will be used for the IoT device and several components of the Google Cloud Platform will form the data pipeline.

![alt text](https://codelabs.developers.google.com/codelabs/iot-data-pipeline/img/b187da70e7caca4d.png)

This part is mainly based on Google Cloud Platform, I am using a existing tutorial for reference.
https://codelabs.developers.google.com/codelabs/iot-data-pipeline/index.html#0

I am following the instructions 1-5 on Google Cloud APIs, and for the security key in my case is

Setup Google Cloud SDK on Raspberry Pi4

In [0]:
curl https://sdk.cloud.google.com | bash

exec -l $SHELL

gcloud init

# 3.Real-time Videostreaming and temperature analysis

I'm using a virtual environment for this guide because it prevents any conflicts between versions of package libraries that may already be installed on your Pi. Keeping it installed in its own environment allows us to avoid this problem.

In [0]:
cd 
#Install virtualenv by issuing:
sudo pip3 install virtualenv

#Then, create the "opt-mask-env" virtual environment by issuing:
python3 -m venv opt-mask-env

#This will create a folder called opt-mask-env inside the directory, the virtual environment folder opt-mask-env will hold all the package libraries for this environment. Next, activate the environment by issuing
source opt-mask-env/bin/activate
#at this point, the command line on your raspberry pi show looks like: (opt-mask-env) pi@raspberrypi:~/Opeartion-Mask-2020 $

#Install libraries for TensorFlow Lite dependencies and OpenCV in requirement.sh, download it to your directory
bash bash requirements.sh
#this 400MB-file downloading takes 15-30mins, go wear a mask while you waiting!

#a build-in video streaming deployed by my self-trained model, under model_lite folder, the model is detect.tflite(rename after my autoML edge-deployed model export in 2.2) and label info is labelmap.txt
python3 TFLite_detection_webcam.py --modeldir=model_lite


python TFLite_detection_image.py --modeldir=model_lite --image=./test/01.jpg
python TFLite_detection_image.py --modeldir=model_lite --image=./Test/01.jpg


[compile_natively_on_raspberry_pi](https://www.tensorflow.org/lite/guide/build_rpi#compile_natively_on_raspberry_pi)

This example uses TensorFlow Lite with Python on a Raspberry Pi to perform real-time image classification using images streamed from the Pi Camera.

Run TensorFlow Lite Object Detection Models on the Raspberry Pi 【Reference】

In [0]:
python3 TFLite_detection_webcam.py --modeldir=model_lite

[TensorFlow Lite Python classification example with Pi Camera](https://github.com/tensorflow/examples/tree/master/lite/examples/image_classification/raspberry_pi)

In [0]:
# Install the TensorFlow Lite runtime
pip3 install https://dl.google.com/coral/python/tflite_runtime-2.1.0-cp37-cp37m-linux_armv7l.whl

In [0]:
python3 classify_picamera.py \
  --model /model-export_iod_tflite-wearing_masks_raspi-2020-01-29T18_02_00.090Z_model.tflite \
  --labels /model-export_iod_tflite-wearing_masks_raspi-2020-01-29T18_02_00.090Z_dict.txt

In [0]:
#Raspberry Pi Python3 Environment
#access API key file, load the credential
export GOOGLE_APPLICATION_CREDENTIALS=cvfinal34944-165d7fd429b3.json

export PROJECT_ID=cvfinal34944

#Enter the following at a command prompt:
curl https://sdk.cloud.google.com | bash

#Restart your shell:
exec -l $SHELL

sudo apt-get install apt-transport-https ca-certificates gnupg

#Run gcloud init to initialize the gcloud environment:
gcloud init

gcloud auth login
gcloud projects add-iam-policy-binding $PROJECT_ID \
   --member="serviceAccount:service-account-name" \
   --role="roles/automl.editor"

raspistill -o image.jpg

python3 predict.py
#python predict.py image.jpg 673576088231 IOD1758457742395179008

# 4 Useful Tools

1. macOS Mojave built-in screen recoder
`
Shift+ Command + 5
`



2. Shut down command
`sudo shutdown -h now`